# DAIOE SSYK2012 Workflow

Organized end-to-end flow for preparing DAIOE + SCB SSYK data.


## 1) Setup Paths and Data Sources

Define imports, workspace paths, and source locations.


In [37]:
import polars as pl
from pathlib import Path


ROOT = Path.cwd().resolve()
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

DAIOE_SOURCE: str = (
    "https://raw.githubusercontent.com/joseph-data/07_translate_ssyk/main/"
    "03_translated_files/daioe_ssyk2012_translated.csv"
)

SCB_SOURCE: str = (
        "https://raw.githubusercontent.com/joseph-data/AI_Econ_daioe_years/daioe_pull/"
        "data/processed/ssyk12_aggregated_ssyk4_to_ssyk1.parquet"
)

## 2) Load DAIOE and SCB Lazily

Read source files as `LazyFrame` objects for efficient transformations.


In [38]:
daioe_lazy_lf = pl.scan_csv(
    DAIOE_SOURCE
)

scb_lazy_lf = pl.scan_parquet(
    SCB_SOURCE
)

## 3) Define Utility Helpers

Create helper functions used for lightweight pipeline inspection.


In [39]:
def inspect_lazy(lf: pl.LazyFrame) -> None:
    """
    Print the shape of a Polars LazyFrame in a memory-efficient manner.

    This function computes the number of rows using a lazy row-count
    aggregation (`pl.len()`) and retrieves the number of columns from
    the resolved schema without materializing the full dataset.

    Parameters
    ----------
    lf : pl.LazyFrame
        The LazyFrame to inspect.

    Notes
    -----
    - The row count triggers execution of the lazy query plan,
      but avoids collecting all columns into memory.
    - The column count is obtained from the schema metadata and
      does not require data materialization.
    - Intended for debugging and validation of large lazy pipelines.
    """
    n_rows = lf.select(pl.len()).collect().item()
    n_cols = len(lf.collect_schema())
    print(f"Rows: {n_rows:,}")
    print(f"Columns: {n_cols}")


## 4) Quick Sanity Checks and Early Military Removal

Preview both sources and remove code-0 military rows early from DAIOE and SCB.


In [40]:
print(daioe_lazy_lf.head(5).collect())

shape: (5, 27)
┌────────────┬──────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ ssyk2012_4 ┆ year ┆ daioe_alla ┆ daioe_stra ┆ … ┆ pctl_rank_ ┆ ssyk2012_ ┆ ssyk2012_ ┆ ssyk2012_ │
│ ---        ┆ ---  ┆ pps        ┆ tgames     ┆   ┆ genai      ┆ 1         ┆ 2         ┆ 3         │
│ str        ┆ i64  ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│            ┆      ┆ f64        ┆ f64        ┆   ┆ f64        ┆ str       ┆ str       ┆ str       │
╞════════════╪══════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 0110 Commi ┆ 2010 ┆ null       ┆ null       ┆ … ┆ null       ┆ 0 Armed   ┆ 01        ┆ 011 Commi │
│ ssioned    ┆      ┆            ┆            ┆   ┆            ┆ forces    ┆ Officers  ┆ ssioned   │
│ armed      ┆      ┆            ┆            ┆   ┆            ┆ occupatio ┆           ┆ armed     │
│ forces…    ┆      ┆            ┆            ┆   ┆            ┆ ns        ┆

In [41]:
print(scb_lazy_lf.head(5).collect())

shape: (5, 7)
┌───────┬───────────┬───────┬───────┬──────┬───────┬─────────────────────────────────┐
│ level ┆ ssyk_code ┆ age   ┆ sex   ┆ year ┆ count ┆ occupation                      │
│ ---   ┆ ---       ┆ ---   ┆ ---   ┆ ---  ┆ ---   ┆ ---                             │
│ str   ┆ str       ┆ str   ┆ str   ┆ i64  ┆ i64   ┆ str                             │
╞═══════╪═══════════╪═══════╪═══════╪══════╪═══════╪═════════════════════════════════╡
│ SSYK4 ┆ 2289      ┆ 35-39 ┆ women ┆ 2015 ┆ 131   ┆ Health professionals not elsew… │
│ SSYK4 ┆ 2135      ┆ 55-59 ┆ men   ┆ 2015 ┆ 299   ┆ Forestry professionals          │
│ SSYK4 ┆ 1292      ┆ 60-64 ┆ women ┆ 2020 ┆ 575   ┆ Administration and service man… │
│ SSYK4 ┆ 8162      ┆ 35-39 ┆ men   ┆ 2016 ┆ 125   ┆ Machine operators, dairy-produ… │
│ SSYK4 ┆ 8213      ┆ 40-44 ┆ women ┆ 2019 ┆ 334   ┆ Metal-, rubber- and plastic-pr… │
└───────┴───────────┴───────┴───────┴──────┴───────┴─────────────────────────────────┘


In [42]:
# daioe_lazy_lf.collect_schema()

In [43]:
## Removed Military Personnel

scb_lazy_lf = scb_lazy_lf.filter(
    pl.col("ssyk_code").str.starts_with("0").not_()
)

daioe_lazy_lf = daioe_lazy_lf.filter(
    pl.col("ssyk2012_4").str.starts_with("0").not_()
)

In [44]:
#scb_lazy_lf.collect().collect_schema()

## 5) Derive SSYK Levels in DAIOE

Split DAIOE SSYK4 into SSYK1-4 and keep SSYK2012-era years.


In [45]:
daioe_lazy_lf_ssyk12 = (
    daioe_lazy_lf\
    .with_columns([
    pl.col("ssyk2012_4").str.slice(0, 1).alias("code_1"),
    pl.col("ssyk2012_4").str.slice(0, 2).alias("code_2"),
    pl.col("ssyk2012_4").str.slice(0, 3).alias("code_3"),
    pl.col("ssyk2012_4").str.slice(0, 4).alias("code_4")
])\
    .drop(pl.col("^ssyk2012.*$"))\
        .filter(pl.col("year") >= 2014) ## The Year stretch from the first SSYK12 publication
)

## 6) Align DAIOE Years to SCB Coverage

Extend DAIOE series forward when SCB has later years.


In [46]:
## Here I extend the years to Latest according to the pulled SCB data (2024, yearly)

base = daioe_lazy_lf_ssyk12

daioe_max = base.select(pl.max("year")).collect().item()
scb_max   = scb_lazy_lf.select(pl.max("year")).collect().item()

missing = list(range(daioe_max + 1, scb_max + 1))

daioe_lazy_lf_extended = (
    base
    if not missing
    else pl.concat(
        [
            base,
            base
            .filter(pl.col("year") == daioe_max)
            .drop("year")
            .join(pl.LazyFrame({"year": missing}), how="cross")
            .select(base.collect_schema().names()),  # ensure same column order/schema
        ],
        how="vertical",
    )
)



In [47]:
inspect_lazy(daioe_lazy_lf_extended)


Rows: 4,686
Columns: 27


## 7) Build SCB SSYK4 Employment Counts

Aggregate SCB counts at year + 4-digit SSYK level.


In [48]:
scb_lazy_lf_level4 = (
    scb_lazy_lf
        .filter(pl.col("ssyk_code").str.len_chars() == 4)
        .group_by(["year", "ssyk_code"])
        .agg(pl.col("count").sum().alias("total_count"))
)



In [49]:
inspect_lazy(scb_lazy_lf_level4)

Rows: 4,686
Columns: 3


## 8) Merge DAIOE with SCB SSYK4 Counts

Join by `year` and 4-digit code, then inspect merged coverage.


In [50]:
daioe_lazy_lf_extended.head(5).collect()

year,daioe_allapps,daioe_stratgames,daioe_videogames,daioe_imgrec,daioe_imgcompr,daioe_imggen,daioe_readcompr,daioe_lngmod,daioe_translat,daioe_speechrec,daioe_genai,pctl_rank_allapps,pctl_rank_stratgames,pctl_rank_videogames,pctl_rank_imgrec,pctl_rank_imgcompr,pctl_rank_imggen,pctl_rank_readcompr,pctl_rank_lngmod,pctl_rank_translat,pctl_rank_speechrec,pctl_rank_genai,code_1,code_2,code_3,code_4
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str
2014,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""1""","""11""","""111""","""1111"""
2015,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""1""","""11""","""111""","""1111"""
2016,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""1""","""11""","""111""","""1111"""
2017,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""1""","""11""","""111""","""1111"""
2018,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""1""","""11""","""111""","""1111"""


In [51]:
scb_lazy_lf_level4.head(5).collect()

year,ssyk_code,total_count
i64,str,i64
2020,"""3321""",11283
2018,"""2143""",27854
2019,"""2652""",4939
2022,"""1120""",20744
2023,"""3424""",2124


In [52]:
daioe_scb_years = daioe_lazy_lf_extended\
    .join(
        scb_lazy_lf_level4,
        left_on=["year", "code_4"],
        right_on=["year", "ssyk_code"],
        how="left"
    )

In [53]:
inspect_lazy(daioe_scb_years)

Rows: 4,686
Columns: 28


## 9) Inspect Unmatched DAIOE Codes

Identify DAIOE rows that have no SCB SSYK4 match.


In [54]:
# DAIOE codes with no SCB match
daioe_scb_years_unmatched = daioe_lazy_lf_extended\
    .join(
        scb_lazy_lf_level4,
        left_on=["year", "code_4"],
        right_on=["year", "ssyk_code"],
        how="anti"
    )



In [55]:
inspect_lazy(daioe_scb_years_unmatched)

Rows: 0
Columns: 27


## 10) Post-Merge Validation

Run quick schema and shape checks on the filtered joined frame before aggregation.


In [56]:
daioe_scb_years.collect_schema()

Schema([('year', Int64),
        ('daioe_allapps', Float64),
        ('daioe_stratgames', Float64),
        ('daioe_videogames', Float64),
        ('daioe_imgrec', Float64),
        ('daioe_imgcompr', Float64),
        ('daioe_imggen', Float64),
        ('daioe_readcompr', Float64),
        ('daioe_lngmod', Float64),
        ('daioe_translat', Float64),
        ('daioe_speechrec', Float64),
        ('daioe_genai', Float64),
        ('pctl_rank_allapps', Float64),
        ('pctl_rank_stratgames', Float64),
        ('pctl_rank_videogames', Float64),
        ('pctl_rank_imgrec', Float64),
        ('pctl_rank_imgcompr', Float64),
        ('pctl_rank_imggen', Float64),
        ('pctl_rank_readcompr', Float64),
        ('pctl_rank_lngmod', Float64),
        ('pctl_rank_translat', Float64),
        ('pctl_rank_speechrec', Float64),
        ('pctl_rank_genai', Float64),
        ('code_1', String),
        ('code_2', String),
        ('code_3', String),
        ('code_4', String),
        ('tot

In [57]:
daioe_scb_years.collect_schema().names()

['year',
 'daioe_allapps',
 'daioe_stratgames',
 'daioe_videogames',
 'daioe_imgrec',
 'daioe_imgcompr',
 'daioe_imggen',
 'daioe_readcompr',
 'daioe_lngmod',
 'daioe_translat',
 'daioe_speechrec',
 'daioe_genai',
 'pctl_rank_allapps',
 'pctl_rank_stratgames',
 'pctl_rank_videogames',
 'pctl_rank_imgrec',
 'pctl_rank_imgcompr',
 'pctl_rank_imggen',
 'pctl_rank_readcompr',
 'pctl_rank_lngmod',
 'pctl_rank_translat',
 'pctl_rank_speechrec',
 'pctl_rank_genai',
 'code_1',
 'code_2',
 'code_3',
 'code_4',
 'total_count']

In [58]:
inspect_lazy(daioe_scb_years)

Rows: 4,686
Columns: 28


## 11) Identify DAIOE Measure Columns

Collect DAIOE metric columns and define the weight expression.


In [59]:
daioe_cols = [
    c for c in daioe_scb_years.collect_schema().names()
    if c.startswith("daioe_")
]

w = pl.col("total_count")


## 12) Manual Aggregation Checks (Legacy)

Compute SSYK3 and SSYK2 weighted/simple aggregates as intermediate checks.


In [60]:

daioe_scb_lv3 = (
    daioe_scb_years
    .select(["code_3", "year", "total_count", *daioe_cols])
    .group_by(["year", "code_3"])
    .agg(
        [
            w.sum().alias("weight_sum"),

            # simple averages
            *[pl.col(c).mean().alias(f"{c}_avg") for c in daioe_cols],

            # employment-weighted averages
            *[
                pl.when(w.sum() > 0)
                  .then((pl.col(c) * w).sum() / w.sum())
                  .otherwise(None)
                  .alias(f"{c}_wavg")
                for c in daioe_cols
            ],
        ]
    )
    .with_columns(pl.lit("SSYK3").alias("level"))
    .rename({"code_3": "ssyk_code"})
)

# preview
daioe_scb_lv3.limit(10).collect()


year,ssyk_code,weight_sum,daioe_allapps_avg,daioe_stratgames_avg,daioe_videogames_avg,daioe_imgrec_avg,daioe_imgcompr_avg,daioe_imggen_avg,daioe_readcompr_avg,daioe_lngmod_avg,daioe_translat_avg,daioe_speechrec_avg,daioe_genai_avg,daioe_allapps_wavg,daioe_stratgames_wavg,daioe_videogames_wavg,daioe_imgrec_wavg,daioe_imgcompr_wavg,daioe_imggen_wavg,daioe_readcompr_wavg,daioe_lngmod_wavg,daioe_translat_wavg,daioe_speechrec_wavg,daioe_genai_wavg,level
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2018,"""154""",612,15.002924,0.23323,2.455084,0.194664,0.063238,0.376393,0.151689,0.101527,0.020088,0.322706,0.764687,15.002924,0.23323,2.455084,0.194664,0.063238,0.376393,0.151689,0.101527,0.020088,0.322706,0.764687,"""SSYK3"""
2019,"""814""",13892,20.416946,0.247975,5.989524,0.236886,0.051135,0.297582,0.094024,0.063005,0.016652,0.199182,0.567461,20.627922,0.251809,6.012463,0.241615,0.052133,0.304849,0.096111,0.064189,0.016862,0.199797,0.580517,"""SSYK3"""
2024,"""336""",19755,27.595808,0.238803,4.507065,0.404282,0.194456,0.535597,0.277399,0.659878,0.026592,0.555079,1.966817,27.595808,0.238803,4.507065,0.404282,0.194456,0.535597,0.277399,0.659878,0.026592,0.555079,1.966817,"""SSYK3"""
2023,"""911""",98156,26.542463,0.219562,4.820361,0.353498,0.177746,0.483403,0.250638,0.627772,0.025151,0.513375,1.825842,25.960116,0.216843,5.306469,0.359797,0.173867,0.440422,0.214433,0.534525,0.021084,0.443041,1.604498,"""SSYK3"""
2021,"""214""",102258,34.795838,0.368817,4.990483,0.574846,0.205431,0.995915,0.290963,0.530543,0.049314,0.48887,2.690565,34.559859,0.365,5.038996,0.561502,0.202131,0.977765,0.287251,0.522901,0.049127,0.485516,2.64555,"""SSYK3"""
2018,"""231""",36446,18.460705,0.272981,2.836045,0.236721,0.079966,0.452173,0.2126,0.136422,0.028643,0.424918,0.95252,18.460705,0.272981,2.836045,0.236721,0.079966,0.452173,0.2126,0.136422,0.028643,0.424918,0.95252,"""SSYK3"""
2019,"""323""",131,20.642328,0.255511,4.721724,0.240212,0.057919,0.342248,0.155825,0.105523,0.026518,0.282899,0.735002,20.642328,0.255511,4.721724,0.240212,0.057919,0.342248,0.155825,0.105523,0.026518,0.282899,0.735002,"""SSYK3"""
2022,"""961""",9320,25.028723,0.219352,4.935848,0.417008,0.145366,0.570048,0.139072,0.438188,0.024164,0.292666,1.710849,25.028723,0.219352,4.935848,0.417008,0.145366,0.570048,0.139072,0.438188,0.024164,0.292666,1.710849,"""SSYK3"""
2022,"""442""",14127,25.776173,0.235574,5.485398,0.377258,0.134319,0.468359,0.147305,0.472316,0.025219,0.309649,1.602905,25.776173,0.235574,5.485398,0.377258,0.134319,0.468359,0.147305,0.472316,0.025219,0.309649,1.602905,"""SSYK3"""


In [61]:
inspect_lazy(daioe_scb_lv3)

Rows: 1,595
Columns: 26


In [62]:

daioe_scb_lv2 = (
    daioe_scb_years
    .select(["code_2", "year", "total_count", *daioe_cols])
    .group_by(["year", "code_2"])
    .agg(
        [
            w.sum().alias("weight_sum"),

            # simple averages
            *[pl.col(c).mean().alias(f"{c}_avg") for c in daioe_cols],

            # employment-weighted averages
            *[
                pl.when(w.sum() > 0)
                  .then((pl.col(c) * w).sum() / w.sum())
                  .otherwise(None)
                  .alias(f"{c}_wavg")
                for c in daioe_cols
            ],
        ]
    )
    .with_columns(pl.lit("SSYK2").alias("level"))
    .rename({"code_2": "ssyk_code"})
)

# preview
daioe_scb_lv2.limit(10).collect()

year,ssyk_code,weight_sum,daioe_allapps_avg,daioe_stratgames_avg,daioe_videogames_avg,daioe_imgrec_avg,daioe_imgcompr_avg,daioe_imggen_avg,daioe_readcompr_avg,daioe_lngmod_avg,daioe_translat_avg,daioe_speechrec_avg,daioe_genai_avg,daioe_allapps_wavg,daioe_stratgames_wavg,daioe_videogames_wavg,daioe_imgrec_wavg,daioe_imgcompr_wavg,daioe_imggen_wavg,daioe_readcompr_wavg,daioe_lngmod_wavg,daioe_translat_wavg,daioe_speechrec_wavg,daioe_genai_wavg,level
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2016,"""42""",64063,15.222916,0.276208,3.016465,0.193958,0.051777,0.333763,0.150161,0.043882,0.011613,0.361039,0.558893,15.665759,0.285982,3.063966,0.202876,0.053947,0.359847,0.15643,0.045232,0.011864,0.359882,0.596326,"""SSYK2"""
2015,"""41""",172538,10.886718,0.265383,3.084224,0.227277,null,0.030132,0.007134,0.03132,0.004615,0.327498,0.102481,11.169447,0.275247,3.179747,0.23383,0.0,0.030623,0.007237,0.031808,0.004668,0.330647,0.104112,"""SSYK2"""
2017,"""81""",107612,13.760842,0.252767,4.082899,0.182171,0.050272,0.238273,0.070593,0.019773,0.006506,0.156713,0.357716,13.949692,0.257314,4.119242,0.184533,0.051074,0.24154,0.0726,0.020249,0.006672,0.160358,0.363368,"""SSYK2"""
2017,"""73""",11947,15.706441,0.289798,4.226192,0.226718,0.0646,0.3383,0.093038,0.02615,0.007967,0.180066,0.501341,15.249059,0.283554,4.081395,0.217682,0.062449,0.32744,0.091482,0.025614,0.00791,0.177351,0.486397,"""SSYK2"""
2014,"""83""",136131,5.302052,0.168206,2.533827,0.13814,null,null,0.00141,0.005651,null,0.02492,0.005651,5.306299,0.169965,2.509589,0.141412,0.0,0.0,0.001507,0.005994,0.0,0.025383,0.005994,"""SSYK2"""
2016,"""53""",550132,11.756115,0.216566,2.937771,0.141045,0.035543,0.22885,0.091249,0.025671,0.006354,0.19941,0.369417,11.882083,0.218794,2.861314,0.143374,0.036725,0.24099,0.09736,0.027331,0.006777,0.209288,0.389986,"""SSYK2"""
2015,"""82""",52315,9.668802,0.206646,3.854413,0.166383,null,0.020069,0.002966,0.012574,0.001819,0.144469,0.05387,9.797718,0.21054,3.905588,0.169686,0.0,0.020566,0.002999,0.012654,0.001821,0.144843,0.054776,"""SSYK2"""
2014,"""24""",167469,5.761664,0.257903,2.032027,0.181268,null,null,0.004584,0.017927,null,0.056609,0.017927,5.664963,0.25131,2.011976,0.177542,0.0,0.0,0.004445,0.017423,0.0,0.055061,0.017423,"""SSYK2"""
2018,"""43""",101964,17.583861,0.275701,3.843336,0.229348,0.066555,0.362864,0.131959,0.086909,0.016745,0.288501,0.709864,17.771307,0.280477,4.043776,0.229315,0.06612,0.354322,0.12894,0.084198,0.01595,0.272323,0.691395,"""SSYK2"""


In [63]:
inspect_lazy(daioe_scb_lv2)

Rows: 473
Columns: 26


## 12b) Optional SSYK1 Check

This inspection assumes `daioe_scb_lv1` exists in the session.


In [64]:
if "daioe_scb_lv1" in globals():
    inspect_lazy(daioe_scb_lv1)
else:
    print("`daioe_scb_lv1` is not defined in this notebook flow; skipping this optional check.")


`daioe_scb_lv1` is not defined in this notebook flow; skipping this optional check.


## 13) Generalized Aggregation + Percentiles

Aggregate SSYK4-1 with a reusable function and add within-year percentiles.


In [65]:
def aggregate_daioe_level(
    lf: pl.LazyFrame,
    code_col: str,
    level_label: str,
    weight_col: str = "total_count",
    prefix: str = "daioe_",
    add_percentiles: bool = True,
    pct_scale: int = 100,
    descending: bool = False,
) -> pl.LazyFrame:

    daioe_cols = [c for c in lf.collect_schema().names() if c.startswith(prefix)]
    w = pl.col(weight_col)

    out = (
        lf
        .group_by(["year", code_col])
        .agg(
            w.sum().alias("weight_sum"),
            pl.col(daioe_cols).mean().name.suffix("_avg"),
            ((pl.col(daioe_cols) * w).sum() / w.sum()).name.suffix("_wavg"),
        )
        .with_columns(pl.lit(level_label).alias("level"))
        .rename({code_col: "ssyk_code"})
    )

    if not add_percentiles:
        return out

    group_keys = ["year", "level"]

    rank_expr = (
        pl.col(f"^{prefix}.*_(avg|wavg)$")
        .rank(method="average", descending=descending)
        .over(group_keys)
    )

    n_expr = pl.len().over(group_keys)

    return out.with_columns(
        (
            pl.when(n_expr > 1)
            .then((rank_expr - 1) / (n_expr - 1))
            .otherwise(0.0)
            * pct_scale
        ).name.prefix("pctl_")
    )


In [66]:
levels = {
    "code_4": "SSYK4",
    "code_3": "SSYK3",
    "code_2": "SSYK2",
    "code_1": "SSYK1",
}

aggregated = [
    aggregate_daioe_level(daioe_scb_years, col, label)
    for col, label in levels.items()
]

daioe_all_levels = (
    pl.concat(aggregated)
    .sort(["level", "year", "ssyk_code"])
)


In [67]:
inspect_lazy(daioe_all_levels)

Rows: 6,853
Columns: 48


In [68]:
print(
    daioe_all_levels
    .group_by("level")
    .len()
    .collect()
)


shape: (4, 2)
┌───────┬──────┐
│ level ┆ len  │
│ ---   ┆ ---  │
│ str   ┆ u32  │
╞═══════╪══════╡
│ SSYK3 ┆ 1595 │
│ SSYK4 ┆ 4686 │
│ SSYK1 ┆ 99   │
│ SSYK2 ┆ 473  │
└───────┴──────┘


## 14) Build 1-5 Level Exposure Columns

Create `daioe_<index>_Level_Exposure` columns from weighted percentile ranks (`pctl_daioe_*_wavg`) using quintile-style bins.


In [69]:
# Convert weighted percentile ranks (0..100) into 1-5 exposure levels
pct_cols = [
    c
    for c in daioe_all_levels.collect_schema().names()
    if c.startswith("pctl_daioe_") and c.endswith("_wavg")
]

exposure_exprs = []
for col_name in pct_cols:
    metric = col_name[len("pctl_daioe_"):-len("_wavg")]
    out_col = f"daioe_{metric}_Level_Exposure"
    p = pl.col(col_name)

    exposure_exprs.append(
        pl.when(p.is_null())
        .then(None)
        .when(p <= 20)
        .then(1)
        .when(p <= 40)
        .then(2)
        .when(p <= 60)
        .then(3)
        .when(p <= 80)
        .then(4)
        .otherwise(5)
        .cast(pl.Int8)
        .alias(out_col)
    )

daioe_all_levels = daioe_all_levels.with_columns(exposure_exprs)

print([c for c in daioe_all_levels.collect_schema().names() if c.endswith("_Level_Exposure")])


['daioe_allapps_Level_Exposure', 'daioe_stratgames_Level_Exposure', 'daioe_videogames_Level_Exposure', 'daioe_imgrec_Level_Exposure', 'daioe_imgcompr_Level_Exposure', 'daioe_imggen_Level_Exposure', 'daioe_readcompr_Level_Exposure', 'daioe_lngmod_Level_Exposure', 'daioe_translat_Level_Exposure', 'daioe_speechrec_Level_Exposure', 'daioe_genai_Level_Exposure']


## 15) Pre-Merge Diagnostics

Inspect schemas and frame shapes before final integration.


In [70]:
daioe_all_levels.collect_schema()

Schema([('year', Int64),
        ('ssyk_code', String),
        ('weight_sum', Int64),
        ('daioe_allapps_avg', Float64),
        ('daioe_stratgames_avg', Float64),
        ('daioe_videogames_avg', Float64),
        ('daioe_imgrec_avg', Float64),
        ('daioe_imgcompr_avg', Float64),
        ('daioe_imggen_avg', Float64),
        ('daioe_readcompr_avg', Float64),
        ('daioe_lngmod_avg', Float64),
        ('daioe_translat_avg', Float64),
        ('daioe_speechrec_avg', Float64),
        ('daioe_genai_avg', Float64),
        ('daioe_allapps_wavg', Float64),
        ('daioe_stratgames_wavg', Float64),
        ('daioe_videogames_wavg', Float64),
        ('daioe_imgrec_wavg', Float64),
        ('daioe_imgcompr_wavg', Float64),
        ('daioe_imggen_wavg', Float64),
        ('daioe_readcompr_wavg', Float64),
        ('daioe_lngmod_wavg', Float64),
        ('daioe_translat_wavg', Float64),
        ('daioe_speechrec_wavg', Float64),
        ('daioe_genai_wavg', Float64),
        

In [71]:
scb_lazy_lf.collect_schema()

Schema([('level', String),
        ('ssyk_code', String),
        ('age', String),
        ('sex', String),
        ('year', Int64),
        ('count', Int64),
        ('occupation', String)])

## Build Age Groups Column
Early Career 1 = Less than 25

In [72]:
scb_lazy_lf.select(pl.col("age").unique()).collect()

age
str
"""16-24"""
"""25-29"""
"""55-59"""
"""30-34"""
"""60-64"""
"""45-49"""
"""40-44"""
"""50-54"""
"""35-39"""


In [73]:
age_map = {
    "16-24": "Early Career 1 (16-24)",
    "25-29": "Early Career 2 (25-29)",
    "30-34": "Developing (30-34)",
    "35-39": "Mid-Career 1 (35-39)",
    "40-44": "Mid-Career 1 (40-44)",
    "45-49": "Mid-Career 2 (45-49)",
    "50-54": "Senior (50+)",
    "55-59": "Senior (50+)",
    "60-64": "Senior (50+)",
    "065-69": "Senior (50+)",
}

scb_lazy_lf = scb_lazy_lf.with_columns(
    pl.col("age").replace(age_map).alias("age_group")
)


In [74]:
inspect_lazy(scb_lazy_lf)

Rows: 137,060
Columns: 8


In [75]:
inspect_lazy(daioe_all_levels)

Rows: 6,853
Columns: 59


## 16) Final Merge

Attach DAIOE aggregates back to the SCB base table.


In [76]:
final_merge = scb_lazy_lf\
    .join(
        daioe_all_levels,
        left_on=["year", "ssyk_code"],
        right_on=["year", "ssyk_code"],
        how="left"
    )

In [77]:
inspect_lazy(final_merge)

Rows: 137,060
Columns: 65


In [40]:
#dd = final_merge.limit(30).collect()

## 17) Export Final Dataset

Write the merged output to parquet for downstream use.


In [78]:
output_path = DATA_DIR / "daioe_scb_years_all_levels.parquet"

final_merge.sink_parquet(output_path)